In [2]:
import derinet.lexicon as dlex
import os
lexicon = dlex.Lexicon()
current_dir = os.getcwd()  # aktualni adresar
file_path = os.path.join(current_dir, "./derinet-2-3.tsv")  #sestaveni cesty
lexicon.load(file_path)

In [17]:
for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma.endswith("ckost"):
        print(lexeme.lemma)

andragogickost
lymfoblastickost
motocyklistickost
elektrofonickost
kvadrofonickost
petrografickost
radiografickost
scintigrafickost
tektogramatickost
androkratickost
antiaristokratickost
aristokratickost
pseudoaristokratickost
autokratickost
antibyrokratickost
byrokratickost
antidemokratickost
demokratickost
polodemokratickost
prodemokratickost
pseudodemokratickost
předdemokratickost
socdemáckost
eurobyrokratickost
idiosynkratickost
meritokratickost
monokratickost
antiplutokratickost
plutokratickost
technokratickost
teokratickost
mammologickost
metrologickost
mineralogickost
pomologickost
sémiologickost
tribologickost
pneumatolytickost
trombolytickost
automatickost
poloautomatickost
telemetrickost
pantomimickost
logopedickost
oftalmopedickost
psychopedickost
sugestopedickost
telepatickost
orthopedickost
ortopedickost
antimonopolistickost
monopolistickost
architektonickost
ablanickost
adriatickost
africkost
křesťanskoafrickost
afrikanistickost
jihoafrickost
vnitroafrickost
východoafrick

    test 8: kontrola přiřazení číslovek podle koncovky (-tý, -tero, -terý)

In [ ]:
def tvorba_predku():
    predci = set()
    for lex in lexicon.iter_lexemes(): 
        if lex.pos == "NUM":
            predci.add(lex.lemma)
    return predci 

def hledani_predku(predci, dite):
    mozni = set()
    for lex in predci:
        if lex in dite: 
            mozni.add(lex)
    
    konecny = set()
    for i in predci:
        for j in predci:
            if i in j and i != j:
                konecny.add(i)
    
    konecny = mozni - konecny

    return konecny

#mireno na slova jako desatero, devátý,
with open("test8.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    predci = tvorba_predku()

    for lexeme in lexicon.iter_lexemes():
        if lexeme.pos == "NUM":
            if lexeme.lemma.endswith("krát"):
                rodic = lexeme.lemma[:-4]
            elif lexeme.lemma.endswith("istý"):
                rodic = lexeme.lemma[:-5]
            elif lexeme.lemma.endswith(("erý", "ero")):
                rodic = lexeme.lemma[:-3]
            elif lexeme.lemma.endswith("ý"):
                rodic = lexeme.lemma[:-1]
            else:
                continue
        
            if rodic.endswith(("desat", "dvacát", "čtyřicát")):
                rodic = rodic[:-2] + "e" + rodic[-1]
            elif rodic.endswith("devat"):
                rodic = rodic[:-2] + "ě" + rodic[-1]
            
            rodice = hledani_predku(predci, rodic)
            chybi = []
            for lex in lexicon.iter_lexemes():
                if lex.lemma in rodice and lex not in lexeme.all_parents: 
                        chybi.append(lexeme.lemma)

            if chybi != []:
                f.write(f"NEJSOU SPOJENI: {lexeme.lemma} s {','.join(chybi)} \n")


test 9: když podstatné jméno ženského rodu končí na 'ka', tak je to odvozené od mužského (ne všechny, ale spoustu z nich -> asistentka x matka)

In [ ]:
with open("test9.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    for lexeme in lexicon.iter_lexemes():
        if lexeme.pos == "NOUN" and lexeme.lemma.endswith("ka"):
            if 'Gender' in lexeme.feats and lexeme.feats['Gender'] == 'Fem':
                predchudce = lexeme.lemma[:-2]
                #veci jako profesorka a profesor
                #ale ne veci jako matka 
                
                if all(p.lemma != predchudce for p in lexeme.all_parents) and len(predchudce) > 3:
                    #ještě musím vyřešit pomolog a pomoložka
                    if predchudce.endswith("ž"): 
                        dalsi = predchudce[:-1] + "g"
                        for lex in lexicon.iter_lexemes(): 
                            if lex.lemma == dalsi and lex.pos == "NOUN":
                                   if 'Gender' in lex.feats and lex.feats['Gender'] == 'Masc':
                                #tohle by melo vyresit i ten problem pro ctyri a ctyrka a motocyklista a motocyklistka
                                #ten problem to neresi, jen si to dělám těžší
                                    f.write(f"{lex.lemma} by měl být rodičem {lexeme.lemma} \n")
                            else: 
                                f.write(f"!!!{lex.lemma} není rodu mužského, ale měl by být rodičem {lexeme.lemma} \n")
                            mame = True
                            break
                    else: 
                        mame = False
                        for lex in lexicon.iter_lexemes():
                            if predchudce == lex.lemma in lex.lemma and lex != lexeme and lex.pos == "NOUN":
                                if 'Gender' in lex.feats and lex.feats['Gender'] == 'Masc':
                                    #tohle by melo vyresit i ten problem pro ctyri a ctyrka a motocyklista a motocyklistka
                                    #ten problem to neresi, jen si to dělám těžší
                                        f.write(f"{lex.lemma} by měl být rodičem {lexeme.lemma} \n")
                                else: 
                                    f.write(f"!!!{lex.lemma} není rodu mužského, ale měl by být rodičem {lexeme.lemma} \n")
                                mame = True
                                break
                            #tady testuju motocyklista a motocyklistka
                            #if predchudce in lex.lemma and lex != lexeme and (len(lex.lemma) - len(predchudce)) == 1 and lex.pos == "NOUN":
                            if predchudce in lex.lemma and lex != lexeme and lex.lemma[:-1] == predchudce and lex.pos == "NOUN":
                                if 'Gender' in lex.feats and lex.feats['Gender'] == 'Masc':
                                    #tohle by melo vyresit i ten problem pro ctyri a ctyrka a motocyklista a motocyklistka
                                    #ten problem to neresi, jen si to dělám těžší
                                        f.write(f"{lex.lemma} by měl být rodičem {lexeme.lemma} \n")
                                else: 
                                    f.write(f"!!!{lex.lemma} není rodu mužského, ale měl by být rodičem {lexeme.lemma} \n")
                                mame = True
                                break
                            
                    if not mame:
                        f.write(f"!!!!!!!!{predchudce}, {lexeme.lemma} co tohle \n")





test 10: podstatné jméno končící na 'ost' by mělo být k něčemu připojeno (jasnost, lenost, prázdnost)

In [ ]:
with open("test10.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    for lexeme in lexicon.iter_lexemes():
        if lexeme.pos == "NOUN" and lexeme.lemma.endswith("ost") and lexeme.lemma != "kost":
            predchudce = lexeme.

test 11: koncovka u podstatného jména 'skost - lidskost

test 12: koncovka končící na 'ckost